# Analyse de données avec Python
## Combiner des DataFrames avec Pandas
Questions
* Comment travailler avec plusieurs sources de données?
* Comment combiner les données de deux DataFrames?

Objectifs
* Combiner les données de plusieurs fichiers en utilisant `concat` et `merge`.
* Combiner deux DataFrames utilisant un identifiant commun.

## Lister des fichiers de données

In [ ]:
# Fonction de "globbing" (recherche par modèle de nom de fichier)
from glob import glob

# Lister une collection de fichiers CSV
fichiers_csv = glob('../data/by_year/*.csv')
fichiers_csv[-5:]

## Concaténer des DataFrames

In [ ]:
import pandas

annee2001 = pandas.read_csv('../data/by_year/surveys_2001.csv')
annee2002 = pandas.read_csv('../data/by_year/surveys_2002.csv')

print(annee2001.shape, annee2002.shape)

In [ ]:
# Concaténer les dataframes verticalement
vertical = pandas.concat([annee2001, annee2002], axis='index')
vertical

In [ ]:
# Réinitaliser l'index du dataframe
# L'option drop=True évite l'ajout d'une colonne avec l'ancien index
vertical = vertical.reset_index(drop=True)
vertical

In [ ]:
# Accumuler les données de tous les fichiers de la collection
liste_df = []

for nom_fichier in glob('../data/by_year/*.csv'):
    df_par_annee = pandas.read_csv(nom_fichier)
    liste_df.append(df_par_annee)

surveys_df = pandas.concat(liste_df, axis='index')
surveys_df = surveys_df.reset_index(drop=True)
surveys_df

## Exercice - Concaténer des DataFrames
* Chargez les données de tous les fichiers CSV du répertoire
  `../data/by_species_id/` et accumulez-les dans une liste.
* Concaténez les DataFrames de cette liste.
* Réinitialisez l'index sans le préserver.

(4 min.)

In [ ]:
liste_df = []

for nom_fichier in glob('../data/by_species_id/*.csv'):
    liste_df.append(pandas.read_csv(nom_fichier))

surveys_sp = pandas.concat(liste_df, axis='index').reset_index(drop=True)
surveys_sp

* Calculez le poids moyen selon l'espèce et le sexe (1 min.)

In [ ]:
# Calculer le poids moyen par espèce et par sexe
poids_espece = surveys_sp.groupby(
    ['species_id', 'sex'])['weight'].mean().unstack()
poids_espece

* Sauvegardez `poids_espece` dans un fichier CSV en préservant
  l'index. Ensuite, rechargez le DataFrame en spécifiant le
  nom de la colonne de l'index (`'species_id'`). (3 min.)

In [ ]:
# Écrire dans un fichier - garder l'index cette fois-ci
fichier_csv = 'poids_par_espece.csv'
poids_espece.to_csv(fichier_csv, index=True)

# Relire les données, fournir le nom de l'index
pandas.read_csv(fichier_csv, index_col='species_id')

## Joindre deux DataFrames

In [ ]:
# Importer un sous-ensemble des espèces pour cet exemple
trois_especes = pandas.read_csv('../data/speciesSubset.csv')
trois_especes

In [ ]:
# Un sous-ensemble des observations
premiers10 = surveys_df.head(10)
premiers10

### Identifier les clés de jonction

In [ ]:
premiers10.columns

In [ ]:
trois_especes.columns

### Une intersection ou "inner join"

![Inner join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/inner-join.png)

In [ ]:
# Calculer l'intersection de premiers10 et trois_especes
cle = 'species_id'
intersection = pandas.merge(
    left=premiers10,
    right=trois_especes,
    left_on=cle,
    right_on=cle
)
# Quelle est la taille de la jonction?
intersection.shape

In [ ]:
intersection

### Jonction de gauche

![Left join of tables A and B](https://datacarpentry.org/python-ecology-lesson/fig/left-join.png)

In [ ]:
jonc_gauche = pandas.merge(
    left=premiers10,
    right=trois_especes,
    how='left',
    on=cle
)
# Quelle est la taille de la jonction?
jonc_gauche.shape

In [ ]:
jonc_gauche

### Les autres types de jonction
* `how='right'` : toutes les lignes du second DataFrame sont gardées
* `how='outer'` : équivalent d'une union, toutes les lignes sont gardées

## Exercice - Joindre toutes les données
`1`. Créez un nouveau DataFrame tel que tous les
enregistrements de `surveys_df` sont gardés dans une jonction
impliquant les informations correspondantes de `species.csv`.
(3 min.)

In [ ]:
species_df = pandas.read_csv('../data/species.csv')

jonc_gauche = pandas.merge(
    left=surveys_df,
    right=species_df,
    how='left',
    on='species_id'
)
jonc_gauche.shape

`2`. Calculez la longueur moyenne des
arrière-pieds (`'hindfoot_length'`) pour chaque genre
d'espèce (`'genus'`) d'une année à l'autre. Transformez le
résultat pour avoir une colonne par genre d'espèce. (4 min.)

In [ ]:
longueurs_moyennes = jonc_gauche.groupby(
    ['year', 'genus'])['hindfoot_length'].mean().unstack()
longueurs_moyennes.tail()

`3`. Calculez le **poids** **moyen** selon le **sexe** pour chaque
**genre d'espèce**. Pour cet exercice, nous allons utiliser un
tableau croisé dynamique à la place de `unstack()`. (3 min.)

In [ ]:
# Utiliser pivot_table() au lieu de groupby() + unstack()
jonc_gauche.pivot_table(
    values='weight',
    index='genus',
    columns='sex',
    aggfunc='mean'
)

## Résumé technique
* **Concaténer** des DataFrames avec `pandas.concat()`
  * Requiert une liste de DataFrames
  * Verticalement si `axis='index'` (par défaut)
  * Horizontalement si `axis='columns'`
  * Réinitialiser l'index au besoin : `reset_index(drop=True)`
* **Joindre** des DataFrames avec `pandas.merge()`
  * `left=`, `right=` : les deux DataFrames à joindre
  * `how=` : `'inner'` (défaut), `'left'`, `'right'`, `'outer'`
  * `left_on=`, `right_on=` : les clés de jonction de chaque DataFrame
  * `on=` : clés de jonction communes aux deux DataFrames